## Summary & Next Steps

### Pipeline Complete!

All processing steps have been executed successfully. The system has:

1. Preprocessed the medical transcript
2. Extracted medical entities (symptoms, diagnoses, treatments)
3. Identified key medical terms
4. Analyzed sentiment and intent
5. Generated SOAP notes
6. Created structured JSON summary
7. Saved all results to output files

### Key Features:
- **No Hardcoded Values**: All parameters are configurable in the first cell
- **Model Selection**: Easy switching between rule-based, ClinicalBERT, and FLAN-T5 models
- **Transformer Integration**: Proper use of transformers library with configurable generation parameters
- **Clean Output**: Consistent JSON format across all models

### Configuration Options:
Change these in the first configuration cell:
- SELECTED_MODEL: Choose your model ('rule-based', 'flan-t5', 'flan-t5-large', 'clinicalbert')
- GENERATION_PARAMS: Adjust temperature, num_beams, max_length for each task
- MODEL_CONFIGS: Update model names and settings
- INPUT_TRANSCRIPT & OUTPUT_DIR: Change file paths

### Next Steps:
1. **Test Different Models**: Change SELECTED_MODEL and re-run
2. **Tune Parameters**: Adjust GENERATION_PARAMS for better results
3. **Add Custom Transcripts**: Update INPUT_TRANSCRIPT path
4. **Compare Models**: Run the comparison cell to test all models
5. **API Integration**: Use the FastAPI endpoint for production

### Production Deployment:
```bash
# Run via command line
python run_demo.py --input path/to/transcript.txt --output results/ --model flan-t5

# Start API server
uvicorn physician_notetaker.api:app --reload
```

In [1]:
# Optional: Compare different models
# This cell allows you to test different models side-by-side

comparison_models = ['rule-based', 'flan-t5', 'clinicalbert']
comparison_results = {}

print("MODEL COMPARISON")
print("="*60)
print("Testing all available models...")
print("Note: This may take several minutes depending on your hardware.\n")

for model_name in comparison_models:
    try:
        print(f"\nTesting {model_name}...")
        
        # Initialize summarizer for this model
        if model_name == 'rule-based':
            test_summarizer = SimpleSummarizer()
        else:
            test_summarizer = LLMSummarizer(model_type=model_name)
        
        # Generate summary
        result = test_summarizer.summarize(
            text=preprocessed['cleaned_text'],
            entities=entities,
            entities_by_type=entities_by_type,
            preprocessed_data=preprocessed
        )
        
        comparison_results[model_name] = result
        print(f"{model_name} complete")
        
    except Exception as e:
        print(f"{model_name} failed: {e}")
        comparison_results[model_name] = {"error": str(e)}

# Display comparison
print("\n" + "="*60)
print("COMPARISON RESULTS:")
print("="*60)
for model_name, result in comparison_results.items():
    print(f"\n{'='*60}")
    print(f"MODEL: {model_name.upper()}")
    print(f"{'='*60}")
    print(json.dumps(result, indent=2, ensure_ascii=False))

MODEL COMPARISON
Testing all available models...
Note: This may take several minutes depending on your hardware.


Testing rule-based...
rule-based failed: name 'SimpleSummarizer' is not defined

Testing flan-t5...
flan-t5 failed: name 'LLMSummarizer' is not defined

Testing clinicalbert...
clinicalbert failed: name 'LLMSummarizer' is not defined

COMPARISON RESULTS:

MODEL: RULE-BASED


NameError: name 'json' is not defined

## Model Comparison (Optional)

Run this cell to compare outputs from different models. Change SELECTED_MODEL in the configuration cell and re-run to test different models.

In [ ]:
# Create output directory if it doesn't exist
output_path = Path(OUTPUT_DIR)
output_path.mkdir(parents=True, exist_ok=True)

# Get base filename from input
base_filename = Path(INPUT_TRANSCRIPT).stem

# Save all results
output_files = {}

# 1. Simple Summary (JSON)
summary_file = output_path / f"{base_filename}.json"
with open(summary_file, 'w', encoding='utf-8') as f:
    json.dump(simple_summary, f, indent=2, ensure_ascii=False)
output_files['summary'] = str(summary_file)

# 2. SOAP Notes (Text)
soap_file = output_path / f"{base_filename}_soap.txt"
with open(soap_file, 'w', encoding='utf-8') as f:
    f.write("SOAP NOTES\n")
    f.write("="*60 + "\n\n")
    f.write("SUBJECTIVE:\n" + soap_note['Subjective']['content'] + "\n\n")
    f.write("OBJECTIVE:\n" + soap_note['Objective']['content'] + "\n\n")
    f.write("ASSESSMENT:\n" + soap_note['Assessment']['content'] + "\n\n")
    f.write("PLAN:\n" + soap_note['Plan']['content'] + "\n")
output_files['soap'] = str(soap_file)

# 3. Entities (JSON)
entities_file = output_path / f"{base_filename}_entities.json"
with open(entities_file, 'w', encoding='utf-8') as f:
    json.dump({
        'total_entities': len(entities),
        'entities_by_type': {k: [e['text'] for e in v] for k, v in entities_by_type.items()},
        'detailed_entities': entities
    }, f, indent=2, ensure_ascii=False)
output_files['entities'] = str(entities_file)

# 4. Keywords (JSON)
keywords_file = output_path / f"{base_filename}_keywords.json"
with open(keywords_file, 'w', encoding='utf-8') as f:
    json.dump(keywords, f, indent=2, ensure_ascii=False)
output_files['keywords'] = str(keywords_file)

# 5. Sentiment & Intent (JSON)
analysis_file = output_path / f"{base_filename}_analysis.json"
with open(analysis_file, 'w', encoding='utf-8') as f:
    json.dump(sentiment_result, f, indent=2, ensure_ascii=False)
output_files['analysis'] = str(analysis_file)

print("All results saved successfully!")
print("="*60)
print("\nOutput Files:")
for name, path in output_files.items():
    print(f"  {name:12s}: {path}")

## Step 7: Save Results to Files

In [ ]:
# Generate simplified summary
print(f"Generating summary using {SELECTED_MODEL} model...")
print("="*60)

simple_summary = summarizer.summarize(
    text=preprocessed['cleaned_text'],
    entities=entities,
    entities_by_type=entities_by_type,
    preprocessed_data=preprocessed
)

print("\nSTRUCTURED SUMMARY (Clean JSON):")
print("="*60)
print(json.dumps(simple_summary, indent=2, ensure_ascii=False))
print("="*60)

## Step 6: Generate Structured Summary (Clean JSON Output)

This generates the final clean JSON output using either rule-based or LLM approach based on the selected model.

In [ ]:
# Generate SOAP notes
soap_note = soap_generator.generate_soap(
    preprocessed['cleaned_text'],
    speaker_turns=preprocessed.get('speaker_turns'),
    entities_by_type=entities_by_type
)

print("SOAP NOTES:")
print("="*60)
print(f"\nSUBJECTIVE:")
print(soap_note['Subjective']['content'])

print(f"\nOBJECTIVE:")
print(soap_note['Objective']['content'])

print(f"\nASSESSMENT:")
print(soap_note['Assessment']['content'])

print(f"\nPLAN:")
print(soap_note['Plan']['content'])

if DISPLAY_CONFIDENCE and 'confidence' in soap_note:
    print(f"\nOverall Confidence: {soap_note['confidence']:.2%}")

## Step 5: SOAP Notes Generation

In [ ]:
# Analyze sentiment and intent
sentiment_result = sentiment_analyzer.classify_transcript(
    preprocessed['cleaned_text'],
    speaker_turns=preprocessed.get('speaker_turns')
)

print("SENTIMENT & INTENT ANALYSIS:")
print("="*60)
print(f"Overall Sentiment: {sentiment_result['overall_sentiment']}")
if DISPLAY_CONFIDENCE:
    print(f"Confidence: {sentiment_result['overall_sentiment_score']:.2f}")

print(f"\nOverall Intent: {sentiment_result['overall_intent']}")

if 'turn_classifications' in sentiment_result and sentiment_result['turn_classifications']:
    print(f"\nPer-turn analysis: {len(sentiment_result['turn_classifications'])} turns analyzed")

## Step 4: Sentiment & Intent Analysis

In [ ]:
# Extract keywords
keywords = keyword_extractor.extract_keywords(
    preprocessed['cleaned_text'],
    entities,
    top_k=10  # Configurable parameter
)

print("KEYWORD EXTRACTION RESULTS:")
print("="*60)
print(f"Top {len(keywords)} keywords extracted:\n")
for i, kw in enumerate(keywords, 1):
    if DISPLAY_CONFIDENCE:
        print(f"{i:2d}. {kw['keyword']:30s} (score: {kw['score']:.3f})")
    else:
        print(f"{i:2d}. {kw['keyword']}")

## Step 3: Keyword Extraction

In [ ]:
# Extract medical entities
ner_result = ner.extract_entities(preprocessed['cleaned_text'])
entities = ner_result['entities']
entities_by_type = ner_result['entities_by_type']

print("NAMED ENTITY RECOGNITION RESULTS:")
print("="*60)
print(f"Total entities extracted: {len(entities)}")
print(f"\nEntities by type:")
for entity_type, entity_list in entities_by_type.items():
    print(f"  {entity_type}: {len(entity_list)} entities")

if DISPLAY_DETAILED_NER:
    print("\n" + "="*60)
    print("DETAILED ENTITIES:")
    print("="*60)
    for entity_type, entity_list in entities_by_type.items():
        if entity_list:
            print(f"\n{entity_type.upper()}:")
            for entity in entity_list[:5]:  # Show first 5 of each type
                conf_str = f" (confidence: {entity.get('confidence', 0):.2f})" if DISPLAY_CONFIDENCE else ""
                print(f"  - {entity['text']}{conf_str}")

## Step 2: Named Entity Recognition (NER)

In [ ]:
# Preprocess the transcript
preprocessed = preprocessor.preprocess(transcript_text)

print("PREPROCESSING RESULTS:")
print("="*60)
print(f"Original length: {len(transcript_text)} chars")
print(f"Cleaned length: {len(preprocessed['cleaned_text'])} chars")
print(f"Speaker turns: {preprocessed['metadata']['num_turns']}")
print(f"  Patient turns: {preprocessed['metadata']['num_patient_turns']}")
print(f"  Physician turns: {preprocessed['metadata']['num_physician_turns']}")
if preprocessed['dates']:
    print(f"Dates found: {len(preprocessed['dates'])}")

## Step 1: Text Preprocessing

In [ ]:
# Load the medical transcript
try:
    with open(INPUT_TRANSCRIPT, 'r', encoding='utf-8') as f:
        transcript_text = f.read()
    
    print(f"Loaded transcript from: {INPUT_TRANSCRIPT}")
    print(f"  Text length: {len(transcript_text)} characters")
    print(f"  Words: {len(transcript_text.split())} words")
    print("\n" + "="*60)
    print("TRANSCRIPT PREVIEW:")
    print("="*60)
    print(transcript_text[:500] + "..." if len(transcript_text) > 500 else transcript_text)
    print("="*60)
    
except FileNotFoundError:
    print(f"Error: Transcript file not found at {INPUT_TRANSCRIPT}")
    print("Please check the file path in the configuration cell")
    raise

## Load Input Transcript

In [ ]:
# Initialize components based on selected model
print(f"Initializing pipeline with {SELECTED_MODEL} model...")

# Step 1: Preprocessor
preprocessor = MedicalPreprocessor()
print("Preprocessor initialized")

# Step 2: NER
ner = MedicalNER(model_name=SPACY_MODEL)
print(f"NER initialized with {SPACY_MODEL}")

# Step 3: Keyword Extractor
keyword_extractor = KeywordExtractor()
print("Keyword Extractor initialized")

# Step 4: Sentiment & Intent Classifier
sentiment_analyzer = SentimentIntentClassifier()
print("Sentiment & Intent Classifier initialized")

# Step 5: SOAP Generator
soap_generator = SOAPGenerator()
print("SOAP Generator initialized")

# Step 6: Summarizer (model-dependent)
if SELECTED_MODEL == 'rule-based':
    summarizer = SimpleSummarizer()
    print("Simple (Rule-based) Summarizer initialized")
else:
    # Map model names
    model_map = {
        'flan-t5': 'flan-t5',
        'flan-t5-large': 'flan-t5-large',
        'clinicalbert': 'clinicalbert'
    }
    
    summarizer = LLMSummarizer(model_type=model_map[SELECTED_MODEL])
    print(f"LLM Summarizer initialized with {SELECTED_MODEL}")

print("\n" + "="*60)
print("All components initialized successfully!")
print("="*60)

## Initialize Pipeline Components

In [ ]:
import sys
import os
import json
import torch
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, os.path.abspath('..'))

# Import project modules
from physician_notetaker.preprocess import MedicalPreprocessor
from physician_notetaker.ner import MedicalNER
from physician_notetaker.keywords import KeywordExtractor
from physician_notetaker.sentiment import SentimentIntentClassifier
from physician_notetaker.soap_generator import SOAPGenerator
from physician_notetaker.simple_summarizer import SimpleSummarizer
from physician_notetaker.llm_summarizer import LLMSummarizer
from physician_notetaker.utils import get_logger

# Initialize logger
logger = get_logger(__name__)

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Libraries imported")
print(f"  Device: {device}")
print(f"  PyTorch version: {torch.__version__}")

## Import Libraries & Initialize System

In [ ]:
# ========== CONFIGURATION PARAMETERS ==========
# All parameters are configurable - no hardcoded values!

# Model Configuration
MODEL_CONFIGS = {
    'clinicalbert': {
        'name': 'emilyalsentzer/Bio_ClinicalBERT',
        'type': 'ner',
        'max_length': 512,
        'description': 'Clinical BERT for medical entity recognition'
    },
    'flan-t5-base': {
        'name': 'google/flan-t5-base',
        'type': 'generation',
        'max_length': 512,
        'description': 'FLAN-T5 Base model for text generation'
    },
    'flan-t5-large': {
        'name': 'google/flan-t5-large',
        'type': 'generation',
        'max_length': 512,
        'description': 'FLAN-T5 Large model for better accuracy'
    }
}

# Generation Parameters (used by LLM models)
GENERATION_PARAMS = {
    'max_input_length': 512,
    'num_beams': 4,
    'temperature': 0.7,
    'do_sample': False,
    'early_stopping': True,
    # Task-specific max lengths
    'patient_name_max_length': 20,
    'symptoms_max_length': 150,
    'diagnosis_max_length': 50,
    'treatment_max_length': 150,
    'status_max_length': 30,
    'prognosis_max_length': 50
}

# Model Selection: Choose which model to use
# Options: 'rule-based', 'clinicalbert', 'flan-t5', 'flan-t5-large'
SELECTED_MODEL = 'rule-based'  # Change this to test different models

# File Paths
INPUT_TRANSCRIPT = '../data/examples/transcript_with_name.txt'  # Path to input transcript
OUTPUT_DIR = '../test_output_notebook'  # Output directory for results

# SpaCy Model (for rule-based approach)
SPACY_MODEL = 'en_core_web_sm'

# Display Configuration
DISPLAY_CONFIDENCE = True  # Show confidence scores in outputs
DISPLAY_DETAILED_NER = True  # Show detailed NER results
VERBOSE_LOGGING = True  # Enable detailed logging

print("Configuration loaded")
print(f"  Selected Model: {SELECTED_MODEL}")
print(f"  Input: {INPUT_TRANSCRIPT}")
print(f"  Output Dir: {OUTPUT_DIR}")

## Configuration & Setup

Configure model parameters and paths here - NO HARDCODED VALUES

# Physician Notetaker: Complete Medical NLP Pipeline

This notebook demonstrates the full physician notetaker system with configurable LLM models (ClinicalBERT, FLAN-T5).

## Features:
- Configurable Models: Choose between rule-based, ClinicalBERT, FLAN-T5-base, or FLAN-T5-large
- Named Entity Recognition (NER): Extract medical entities with confidence scores
- Structured Summaries: Generate clean JSON output
- Keyword Extraction: Identify important medical terms
- Sentiment & Intent Analysis: Understand patient communication
- SOAP Notes: Clinical documentation (Subjective, Objective, Assessment, Plan)

## Output Format:
```json
{
  "Patient_Name": "Janet Jones",
  "Symptoms": ["Headache", "Neck pain", "Back pain"],
  "Diagnosis": "Whiplash injury",
  "Treatment": ["Rest", "Pain medication", "Physical therapy"],
  "Current_Status": "Patient is in moderate discomfort",
  "Prognosis": "Expected to recover with proper treatment"
}
```